# 데이터 전처리

### 목차
(1) 데이터 및 라이브러리 준비

(2) 전처리

1. 결측치 제거
2. 배급사 전처리
3. 배급사별 관객수 중위값으로 랭킹 인코딩 `distributor_rank`
4. 장르별 관객수 평균값으로 랭킹 인코딩 `gerne_rank`
5. 주연 배우들이 출연한 영화 관객수의 평균 중 max관객수 `actor_max_num`, `actor_max`
6. 개봉 계절 `season` , 휴일 여부 `holiday`
7. 개봉 연도 & 월 & 요일 데이터 `release_time_year`, `release_time_month`, `release_time_weekday`  
    
(3) 최종 데이터 확인

# (1) 데이터 및 라이브러리 준비

### > 라이브러리

In [ ]:
# pytimekr 라이브러리 설치
!pip install pytimekr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
from pytimekr import pytimekr

### > 데이터 불러오기
- 전처리 할 데이터 파일 : 'train_merge_2.csv', 'test_merge_2.csv' 

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print(IN_COLAB)

True


In [ ]:
if IN_COLAB:
  print("colab mode")
  from google.colab import drive
  drive.mount("/content/working_drive")
  train_file = '/content/working_drive/MyDrive/빅예분4팀 /dataset/train_merge_2.csv'
  test_file = '/content/working_drive/MyDrive/빅예분4팀 /dataset/test_merge_2.csv'
  train_final_file = '/content/working_drive/MyDrive/빅예분4팀 /dataset/train_final.csv'
  test_final_file = '/content/working_drive/MyDrive/빅예분4팀 /dataset/test_final.csv'
else:
  print("not colab mode")
  train_file = './dataset/train_merge_2.csv'
  test_file = './dataset/test_merge_2.csv'
  train_final_file = './dataset/train_final.csv'
  test_final_file = './dataset/test_final.csv'


colab mode
Drive already mounted at /content/working_drive; to attempt to forcibly remount, call drive.mount("/content/working_drive", force_remount=True).


In [ ]:
# 데이터 불러오기
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

In [ ]:
train_data.head(5)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,actors,review_cnt
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,김무열|진선규,134
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501,이병헌|조승우|백윤식,2779
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083,김수현|박기웅|이현우,3028
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.0,2,20,6,217866,윤제문|송하윤|성준,248
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.0,1,251,2,483387,임창정|엄지원,229


# (2) 전처리

## 1. 결측치 제거 
- `pd.DataFrame.info()` : 컬럼별 데이터 타입 확인
- `pd.DataFrame.isnull().sum()` : Series별 결측치 갯수를 구합니다.
- `pd.DataFrame.fillna()`
    - 결측치를 채우고자 하는 column과 결측치를 대신하여 넣고자 하는 값을 명시해주어야 합니다.
    - 범주형 변수일 경우, 최빈값으로 대체할 수 있습니다. 

#### 결측치 제거

In [ ]:
train_data.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
actors              2
review_cnt          0
dtype: int64

In [ ]:
test_data.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
actors              2
review_cnt          0
dtype: int64

In [ ]:
# 결측치가 있는 행의 dir_prev_num이 모두 0인지 확인하기 
train_data[train_data['dir_prev_bfnum'].isna()]['dir_prev_num'].sum()

0

In [ ]:
test_data[test_data['dir_prev_bfnum'].isna()]['dir_prev_num'].sum()

0

In [ ]:
# dir_prev_bfnum Na값 0으로 변환
train_data['dir_prev_bfnum'].fillna(0, inplace=True)
test_data['dir_prev_bfnum'].fillna(0, inplace=True)

In [ ]:
# actors 결측치 제거 
train_data['actors'].fillna('No Data', inplace=True)
test_data['actors'].fillna('No Data', inplace=True)

In [ ]:
# 다큐멘터리 actors 오류값 처리 
train_data.loc[(train_data["actors"].str.contains("관람가")), "actors"] = 'No Data'
test_data.loc[(test_data["actors"].str.contains("관람가")), "actors"] = 'No Data'

In [ ]:
test_data


,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,actors,review_cnt
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,300529.0,2,304,3,설경구|류승범|한혜진,536
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,342700.2,4,275,3,이나영|김지석|김희수,221
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4206610.7,3,419,7,김윤진|나문희|강예원,1188
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,691342.0,2,408,2,송강호|강동원,1292
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,31738.0,1,380,1,지진희,388
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2590.0,1,4,4,신연우|김가현|김호원,21
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,0.0,0,18,2,도꾜 조선 중고급학교 권투부|김상수,32
240,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,0.0,0,224,4,윤소이|신현빈|임서주,113
241,말하지 못한 비밀,(주)씨타마운틴픽쳐스,드라마,2015-10-22,102,청소년 관람불가,송동윤,50699.0,1,68,7,지수|신재승|우주원,28


In [ ]:
train_data.isna().sum()

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
actors            0
review_cnt        0
dtype: int64

In [ ]:
test_data.isna().sum()

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
actors            0
review_cnt        0
dtype: int64

## 2. 배급사 전처리


In [ ]:
def get_distributor(x) :
    x = x.replace("(주)","")
    x = re.sub(r'[^0-9a-zA-Z가-힣]', '', x)
    
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [ ]:
train_data['distributor'] = train_data.distributor.apply(get_distributor)
test_data['distributor'] = test_data.distributor.apply(get_distributor)

In [ ]:
train_data.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,actors,review_cnt
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,2,23398,김무열|진선규,134
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501,이병헌|조승우|백윤식,2779
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083,김수현|박기웅|이현우,3028


## 3. 배급사별 관객수 중위값으로 랭킹 인코딩 `distributor_rank`
- train_data : 총 147개의 배급사 

In [ ]:
tr_nm_rank  = train_data.groupby('distributor').box_off_num.median().reset_index(name='distributor_rank').sort_values(by='distributor_rank')
tr_nm_rank

,distributor,distributor_rank
109,인피니티엔터테인먼트,2.0
15,고구마공작소,8.0
51,사람과사람들,42.0
96,위드시네마,46.0
19,나우콘텐츠,54.0
...,...,...
111,전망좋은영화사,1214237.0
104,이십세기폭스코리아,1422844.0
55,쇼박스,2138560.0
83,영구아트무비,2541603.0


In [ ]:
tr_nm_rank['distributor_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]
tr_nm_rank

,distributor,distributor_rank
109,인피니티엔터테인먼트,1
15,고구마공작소,2
51,사람과사람들,3
96,위드시네마,4
19,나우콘텐츠,5
...,...,...
111,전망좋은영화사,143
104,이십세기폭스코리아,144
55,쇼박스,145
83,영구아트무비,146


In [ ]:
train_data = pd.merge(train_data, tr_nm_rank, how = 'left')
test_data = pd.merge(test_data, tr_nm_rank, how = 'left')

In [ ]:
train_data.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum      0
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
actors              0
review_cnt          0
distributor_rank    0
dtype: int64

In [ ]:
train_data.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,actors,review_cnt,distributor_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,2,23398,김무열|진선규,134,134
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501,이병헌|조승우|백윤식,2779,145
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083,김수현|박기웅|이현우,3028,145


In [ ]:
test_data.isna().sum()

title                0
distributor          0
genre                0
release_time         0
time                 0
screening_rat        0
director             0
dir_prev_bfnum       0
dir_prev_num         0
num_staff            0
num_actor            0
actors               0
review_cnt           0
distributor_rank    31
dtype: int64

#### train 데이터 기준으로 배급사 랭크를 계산해서 test 데이터에만 존재하는 배급사의 경우 랭크가 누락될 수 있다. 
-> 따라서 test_data에는 결측값이 존재할 수 있다.

-> train_data['distributor_rank'].mean() : train_data의 'distributor_rank'평균값으로 결측값 채우기

In [ ]:
# 결측값이 존재함
test_data[test_data['distributor_rank'].isna()]

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,actors,review_cnt,distributor_rank
5,회오리 바람,모쿠슈라,드라마,2010-02-25,95,15세 관람가,장건재,0.0,0,156,2,서준영|이민지,51,NaN
7,이웃집 남자,루믹스미디어,드라마,2010-03-18,100,청소년 관람불가,장동홍,0.0,0,117,5,윤제문|서태화|김인,69,NaN
9,반가운 살인자,롯데쇼핑롯데엔터테인먼트,코미디,2010-04-08,107,15세 관람가,김동욱,0.0,0,255,3,유오성|김동욱|심은경,284,NaN
12,작은 연못,노근리프로덕션,액션,2010-04-15,86,15세 관람가,이상우,34.0,1,310,9,문성근|전혜진|신명철,227,NaN
20,인플루언스,리얼라이즈픽쳐스,드라마,2010-08-19,78,15세 관람가,이재규,0.0,0,10,7,이병헌|한채영|조재현,53,NaN
24,탈주,청년필름,드라마,2010-09-02,110,청소년 관람불가,이송희일,0.0,0,11,3,이영훈|소유진|김지한,33,NaN
29,우리 만난 적 있나요,리아코어콘텐츠,멜로/로맨스,2010-11-25,100,12세 관람가,임진평,1286.0,1,137,2,박재정|윤소이,22,NaN
39,REC 알이씨,핑크로봇필름,멜로/로맨스,2011-11-24,65,청소년 관람불가,소준문,0.0,0,39,2,송재하|조혜훈,39,NaN
40,량강도 아이들,CinemaSAM,드라마,2011-11-17,95,전체 관람가,김성훈,163194.0,1,131,9,김환영|주혜리|신민규,29,NaN
46,은어,갑종필름,드라마,2011-09-29,95,15세 관람가,박갑종,0.0,0,31,7,이지혁|홍예나|김추련,1,NaN


In [ ]:
# train_data의 'distributor_rank'평균값으로 결측값 채우기
test_data['distributor_rank'] = test_data['distributor_rank'].fillna(train_data['distributor_rank'].mean())
test_data['distributor_rank'] = pd.to_numeric(test_data['distributor_rank'])

In [ ]:
test_data.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum      0
dir_prev_num        0
num_staff           0
num_actor           0
actors              0
review_cnt          0
distributor_rank    0
dtype: int64

## 4. 장르별 관객수 평균값으로 랭킹 인코딩 `gerne_rank`
- 1~12점
- 뮤지컬 : 1 / 다큐멘터리 : 2 / 서스펜스 : 3 / 애니메이션 : 4 / 멜로/로맨스 : 5 / 미스터리 : 6
- 공포 : 7 / 드라마 : 8 / 코미디 : 9 / SF : 10 / 액션 : 11 / 느와르 : 12

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
# 장르별 관객수 평균값
train_data.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬         6627.0
다큐멘터리      67172.3
서스펜스       82611.0
애니메이션     181926.7
멜로/로맨스    425968.0
미스터리      527548.2
공포        590832.5
드라마       625689.8
코미디      1193914.0
SF       1788345.7
액션       2203974.1
느와르      2263695.1
Name: box_off_num, dtype: float64

In [ ]:
train_data['genre_rank'] = train_data.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test_data['genre_rank'] = test_data.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

In [ ]:
train_data.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,actors,review_cnt,distributor_rank,genre_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,2,23398,김무열|진선규,134,134,11
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501,이병헌|조승우|백윤식,2779,145,12
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083,김수현|박기웅|이현우,3028,145,11


## 5. 주연 배우들이 출연한 영화 관객수의 평균 중 max관객수, max관객을 가진 배우이름 `actor_max_num`, `actor_max`
- 각 배우의 관객수 평균 중 max값 구하기 

In [ ]:
# 각 배우의 평균 관객수
actors_list = list(set("|".join(list(train_data['actors'])).split("|"))) # 중복제거한 전체 배우 목록
print(len(actors_list)) # 배우 수

1070


In [ ]:
# 배우별 평균 관객수
actors_dict = dict()
for actor in actors_list:
    box_off_num_mean = train_data[train_data["actors"].str.contains(actor)]["box_off_num"].mean()
    actors_dict[actor] = box_off_num_mean
#print(actors_dict)

#### train 데이터 기준으로 배우 정보를 불러와서, test 데이터에만 존재하는 배우 관련 데이터가 누락될 수 있다.
-> 따라서 test_data에는 결측값이 존재할 수 있다.

-> 배우 정보가 없는 경우 **장르별 관객수 평균값**으로 채운다 

In [ ]:
# 장르별 평균 관객수
genre_dict = dict(train_data.groupby('genre').box_off_num.mean().sort_values())
genre_dict

{'뮤지컬': 6627.0,
 '다큐멘터리': 67172.25806451614,
 '서스펜스': 82611.0,
 '애니메이션': 181926.66666666666,
 '멜로/로맨스': 425968.03846153844,
 '미스터리': 527548.1764705882,
 '공포': 590832.5238095238,
 '드라마': 625689.7918552036,
 '코미디': 1193914.0,
 'SF': 1788345.6923076923,
 '액션': 2203974.0714285714,
 '느와르': 2263695.111111111}

In [ ]:
# 각 배우의 관객수 중 max관객수 `actor_max_num`
def get_actor_max_num(row):
    
    actors = row['actors'].split("|")
    val = []
    try:
        for actor in actors:
            val.append(actors_dict[actor])
            #print(actor)
        #print(val)
        return max(val)  
    except:
        return genre_dict[row['genre']]

In [ ]:
# 관객수 max인 배우 `main_actor`
def get_main_actor(x):
    actors = x.split("|")
    val = dict()
    try:
        for actor in actors:
            val[actor]= actors_dict[actor]
            #print(actor)
        #print(max(val, key=val.get) )
            # 관객수 가장 많은 배우이름 출력
            return max(val, key=val.get)  
    except:
            # 관객수 데이터 없는 경우 첫번째 배우 출력
            return actors[0]  


In [ ]:
train_data['actor_max_num'] = train_data[["actors", "genre"]].apply(get_actor_max_num, axis=1)

In [ ]:
train_data["actor_max"]= train_data.actors.apply(get_main_actor)

In [ ]:
train_data.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,actors,review_cnt,distributor_rank,genre_rank,actor_max_num,actor_max
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,2,23398,김무열|진선규,134,134,11,3752789.0,김무열
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501,이병헌|조승우|백윤식,2779,145,12,4745863.8,이병헌
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083,김수현|박기웅|이현우,3028,145,11,3487385.0,김수현


In [ ]:
# test data
test_data['actor_max_num'] = test_data[["actors", "genre"]].apply(get_actor_max_num, axis=1)

In [ ]:
test_data["actor_max"]= test_data.actors.apply(get_main_actor)

In [ ]:
test_data.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,actors,review_cnt,distributor_rank,genre_rank,actor_max_num,actor_max
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,300529.0,2,304,3,설경구|류승범|한혜진,536,132.0,12,4291705.3,설경구
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,342700.2,4,275,3,이나영|김지석|김희수,221,145.0,5,425968.0,이나영
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4206610.7,3,419,7,김윤진|나문희|강예원,1188,141.0,8,5907481.3,김윤진


In [ ]:
test_data.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum      0
dir_prev_num        0
num_staff           0
num_actor           0
actors              0
review_cnt          0
distributor_rank    0
genre_rank          0
actor_max_num       0
actor_max           0
dtype: int64

## 6. 개봉 계절 `season` , 휴일 여부 `holiday`
- `season`
    - 겨울(1) 12 ~ 2월, 봄(2) 3 ~ 5월, 여름(3) 6 ~ 8월, 가을(4) 9 ~ 11월 
-  `holiday`
   - pytimekr 라이브러리 : 대한민국 공휴일 (ex. 한글날, 어린이날, 현충일, 광복절, 제현절 등.. 불러옴)
   - 토요일, 일요일 포함
   - True: 1, False:0
 

In [ ]:
seasons = {(12, 1, 2): 1, (3, 4, 5): 2, (6, 7, 8): 3, (9, 10, 11): 4}
def season(ser):
    for k in seasons.keys():
        if ser in k:
            return seasons[k]

In [ ]:
def get_holidays(min, max):
    list_holidays = []
    for year in range(min,max+1):
        list_holidays = list_holidays + pytimekr.holidays(year) 
    # datetime.date 형식으로 반환되어 string으로 다시 변환한다.
    list_holidays = [i.strftime("%Y-%m-%d") for i in list_holidays]
    return list_holidays

## 7. 개봉 연도 & 월 & 요일 데이터 `release_time_year`, `release_time_month`, `release_time_weekday`
- `release_time_weekday` : 요일숫자(0-월, 1-화) (=dayofweek)

In [ ]:
def merge_dates(data):
    data['release_time_dt'] = pd.to_datetime(data['release_time'], format='%Y-%m-%d', errors='raise')
    data['release_time_month']  = data['release_time_dt'].dt.month
    data['release_time_year']  = data['release_time_dt'].dt.year
    data['release_time_weekday']       = data['release_time_dt'].dt.weekday       # 요일숫자(0-월, 1-화) (=dayofweek)
    data['season'] = data['release_time_dt'].dt.month.apply(season)
    list_holidays = get_holidays(data['release_time_year'].min(),data['release_time_year'].max())
    data['holiday'] = (data['release_time'].isin(list_holidays) | (data['release_time_weekday']>4)).astype(int)
    data.drop(labels = ['release_time_dt'], axis = 1, inplace=True)
    return data

In [ ]:
train_data = merge_dates(train_data)
test_data = merge_dates(test_data)

In [ ]:
train_data[train_data['holiday']==True]

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,...,review_cnt,distributor_rank,genre_rank,actor_max_num,actor_max,release_time_month,release_time_year,release_time_weekday,season,holiday
19,하하하,스폰지,드라마,2010-05-05,115,청소년 관람불가,홍상수,0.0,0,74,...,233,40,8,1803565.0,김상경,5,2010,2,2,1
94,렛 미 아웃,백두대간,드라마,2013-08-15,97,12세 관람가,소재영,922680.0,1,29,...,29,108,8,1597.5,권현상,8,2013,3,3,1
109,카페 서울,나이너스엔터테인먼트,드라마,2010-01-02,94,12세 관람가,타케 마사하루,0.0,0,41,...,12,109,8,20971.3,김정훈,1,2010,5,1,1
113,"열여덟, 열아홉",키노아이,멜로/로맨스,2012-03-01,94,15세 관람가,배광수,817317.7,3,96,...,64,66,5,878341.0,유연석,3,2012,3,2,1
153,첫사랑 열전,웃기씨네,멜로/로맨스,2010-05-05,114,15세 관람가,박범훈,0.0,0,136,...,40,20,5,213930.6,이청아,5,2010,2,2,1
247,마이 플레이스,KTG상상마당,다큐멘터리,2014-01-30,77,전체 관람가,박문칠,399.0,2,17,...,47,79,2,2951.0,박문칠,1,2014,3,1,1
257,러시안 소설,KTG상상마당,드라마,2013-09-19,140,15세 관람가,신연식,0.0,0,29,...,96,79,8,5992.0,강신효,9,2013,3,4,1
288,타워,CJ,드라마,2012-12-25,121,12세 관람가,김지훈,2849087.5,4,126,...,1597,141,8,4655728.2,설경구,12,2012,1,1,1
342,그리고 싶은 것,시네마달,다큐멘터리,2013-08-15,92,전체 관람가,권효,891.0,1,24,...,96,72,2,5014.0,권윤덕,8,2013,3,3,1
373,"앵두야, 연애하자",홀리가든,드라마,2013-06-06,98,15세 관람가,정하린,0.0,0,34,...,102,61,8,213930.6,류현경,6,2013,3,3,1


# (3) 최종 데이터 확인

In [ ]:
train_data.info()
train_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 600 non-null    object 
 1   distributor           600 non-null    object 
 2   genre                 600 non-null    object 
 3   release_time          600 non-null    object 
 4   time                  600 non-null    int64  
 5   screening_rat         600 non-null    object 
 6   director              600 non-null    object 
 7   dir_prev_bfnum        600 non-null    float64
 8   dir_prev_num          600 non-null    int64  
 9   num_staff             600 non-null    int64  
 10  num_actor             600 non-null    int64  
 11  box_off_num           600 non-null    int64  
 12  actors                600 non-null    object 
 13  review_cnt            600 non-null    int64  
 14  distributor_rank      600 non-null    int64  
 15  genre_rank            6

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,...,review_cnt,distributor_rank,genre_rank,actor_max_num,actor_max,release_time_month,release_time_year,release_time_weekday,season,holiday
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,...,134,134,11,3752789.0,김무열,11,2012,3,4,0
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,...,2779,145,12,4745863.8,이병헌,11,2015,3,4,0
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,...,3028,145,11,3487385.0,김수현,6,2013,2,3,0
3,나는 공무원이다,NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.0,2,20,...,248,140,9,679544.0,윤제문,7,2012,3,3,0
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.0,1,251,...,229,145,9,661605.8,임창정,11,2010,3,4,0


In [ ]:
test_data.info()
test_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 243 non-null    object 
 1   distributor           243 non-null    object 
 2   genre                 243 non-null    object 
 3   release_time          243 non-null    object 
 4   time                  243 non-null    int64  
 5   screening_rat         243 non-null    object 
 6   director              243 non-null    object 
 7   dir_prev_bfnum        243 non-null    float64
 8   dir_prev_num          243 non-null    int64  
 9   num_staff             243 non-null    int64  
 10  num_actor             243 non-null    int64  
 11  actors                243 non-null    object 
 12  review_cnt            243 non-null    int64  
 13  distributor_rank      243 non-null    float64
 14  genre_rank            243 non-null    int64  
 15  actor_max_num         2

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,...,review_cnt,distributor_rank,genre_rank,actor_max_num,actor_max,release_time_month,release_time_year,release_time_weekday,season,holiday
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,300529.0,2,304,...,536,132.0,12,4291705.3,설경구,1,2010,3,1,0
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,342700.2,4,275,...,221,145.0,5,425968.0,이나영,1,2010,3,1,0
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4206610.7,3,419,...,1188,141.0,8,5907481.3,김윤진,1,2010,3,1,0
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,691342.0,2,408,...,1292,145.0,11,6449057.8,송강호,2,2010,3,1,0
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,31738.0,1,380,...,388,141.0,7,51207.0,지진희,2,2010,3,1,0


### 파일로 내보내기

In [ ]:
train_data.to_csv(train_final_file,index=False)
test_data.to_csv(test_final_file,index=False)

### 최종 변수
1. title
2. distributor
3. genre
4. release_time
5. time
6. screening_rat
7. director
8. dir_prev_bfnum
9. dir_prev_num
10. num_staff
11. num_actor
12. **`actors`** :  네이버 영화 배우 정보
13. **`review_cnt`** : 네이버 영화 상세 페이지의 리뷰 개수
14. **`distributor_rank`** : 배급사의 영향력 정도(관객수가 높은 배급사일수록 값이 큼, min:1, max:147)
15. **`genre_rank`** : 장르의 영향력 정도(관객수가 높은 장르일수록 값이 큼, min:1, max:12)
16. **`actor_max_num`** : 주연배우들의 평균 관객수 중 가장 높은 관객수
17. **`actor_max`** : 주연배우들의 평균 관객수 중 가장 높은 관객수를 가진 배우
18. **`release_time_month`** : 개봉월
19. **`release_time_year`** : 개봉년도
20. **`release_time_weekday`** : 개봉요일
21. **`season`** : 개봉계절
22. **`holiday`**  : 개봉일 휴일여부



### 크롤링 & 전처리로 새롭게 추가된 변수(총 11개)

: actors / review_cnt / distributor_rank / genre_rank / actor_max_num / actor_max / release_time_month / release_time_year / release_time_weekday / season  / holiday